In [1]:
import pandas as pd
import numpy as np
import os
from glob import glob
from collections import Counter
os.chdir('/Users/xiaodiu/Documents/github/JudgeSentences/')

In [5]:
df = pd.DataFrame()
itr = pd.read_stata('rawdata/sentences/opafy00nid.dta',iterator=True)
chunk = itr.get_chunk(10000)
df = df.append(chunk, ignore_index=True)

In [4]:

from glob import glob

import pandas as pd
import re


folders = glob('*')

num = 0
for folder in folders:        
    members = glob(folder + '/*txt')
    num += len(members)
    
print "total %d" %(num)

total 251963


In [15]:
case2judge = pd.read_pickle('rawdata/cluster2songername.pkl')
casefreq_akd = pd.read_pickle('rawdata/case_level/casefreqs-akd.pkl')
id2word = pd.read_pickle('rawdata/id2word-2.pkl')
word2id = pd.read_pickle('rawdata/word2id-2.pkl')
df_Holger = pd.read_stata('rawdata/Holger_matched_USSC_TRAC2.dta')
df_Holger_demean = pd.read_stata('rawdata/Holger_matched_USSC_TRAC2_include_demean.dta')
df_Trac_USSC_Judges = pd.read_stata('rawdata/Trac_USSC_JudgesBIOs.dta')
docfreq_akd = pd.read_pickle('rawdata/text/docfreqs-akd.pkl')

In [2]:
case2judge = pd.read_pickle('rawdata/cluster2songername.pkl')

In [3]:
case2judge

{2097152: 'NORGLE, CHARLES R., SR.',
 1572868: 'WILLIAMS, RICHARD L.',
 2097157: 'BRODY, MORTON A.',
 1572871: 'WEIGEL, STANLEY A.',
 2097162: 'LEE, WILLIAM CHARLES',
 1572875: 'MAGNUSON, PAUL A.',
 2097168: 'SQUATRITO, DOMINIC J.',
 1660248: 'REDDEN, JAMES ANTHONY, JR.',
 2097172: 'TORRES, ERNEST C.',
 2097175: 'KATZ, MARVIN',
 1572889: 'CURTIN, JOHN T.',
 1572894: 'QUACKENBUSH, JUSTIN LOWE',
 2097186: 'MCAVOY, THOMAS J.',
 1572899: 'SIFTON, CHARLES P.',
 2097191: 'FRYE, HELEN JACKSON',
 1572904: 'SHADUR, MILTON IRVING',
 2097198: "O'KELLEY, WILLIAM C.",
 2097203: 'PAYNE, ROBERT ELKIN',
 2097211: 'WARDLAW, KIM MCLANE',
 1572925: 'BUA, NICHOLAS J.',
 2097218: 'WINGATE, HENRY T.',
 1466481: 'SHARP, GEORGE KENDALL',
 1572938: 'PETTINE, RAYMOND J.',
 2097230: 'WINGATE, HENRY T.',
 1572943: 'IRIZARRY, DORA L.',
 1922404: 'HARVEY, ALEXANDER, II',
 1572955: 'DALY, T.F. GILROY',
 2097245: 'RANDA, RUDOLPH T.',
 1572960: 'JOYNER, JAMES CURTIS',
 1572962: 'SHAW, JOHN M.',
 2097253: 'GONZALEZ, IR

In [114]:
case2judge = pd.read_pickle('rawdata/cluster2songername.pkl')
judge2case = dict(zip(case2judge.values(),case2judge.keys()))
files = glob('rawdata/case_level/*.pkl')
num = len(files)
judge2count_bigram = Counter()
i = 1
for file in files:
    #print('file :'+ file)
    case_freq = pd.read_pickle(file)
    for k,v in case_freq.items():
        k = int(k)
        if k in case2judge:
            judge = case2judge[k]
            #print('judge_name:' + judge)
            if(judge2count_bigram[judge] == 0):
                judge2count_bigram[judge] = Counter()
            judge2count_bigram[judge] += v
    if(i % 20 == 0 ):
        print('finished : %s' % (i))
    i += 1

finished : 20
finished : 40
finished : 60
finished : 80
finished : 100


In [118]:
judge2exist_bigram = Counter()
for k, v in judge2count_bigram.items():
    judge2exist_bigram[k] = Counter(v.keys())

$$
y_i = \alpha Judge_i + \beta X + \alpha_2type + \epsilon_i
$$

In [120]:
pd.to_pickle(judge2exist_bigram,
             'rawdata/judge2exist_bigram.pkl')
pd.to_pickle(judge2count_bigram,
             'rawdata/judge2count_bigram.pkl')

In [122]:
sentence = pd.read_pickle('rawdata/senttot_matched.pkl')

In [123]:
sentence

,judgeid,demean_senttot
0,"CARTER, GENE",-75.735191
2,"CARTER, GENE",-11.735191
4,"SINGAL, GEORGE Z.",-62.735191
5,"CARTER, GENE",-35.735191
6,"WOODCOCK, JOHN A., JR.",-64.705193
8,"SINGAL, GEORGE Z.",-70.735191
10,"CARTER, GENE",-13.735191
11,"CARTER, GENE",41.264809
13,"WOODCOCK, JOHN A., JR.",-75.845192
14,"WOODCOCK, JOHN A., JR.",-35.735191


In [124]:
df_Trac_USSC_Judges = pd.read_stata('rawdata/Trac_USSC_JudgesBIOs.dta')

In [133]:
df_Trac_USSC_Judges.head()
for i in range(10):
    print(i)
    print(df_Trac_USSC_Judges.columns[i:(1+i)*20])

0
Index(['judgefirstname', 'judgelastname', 'judgemiddlename', 'pooffice',
       'usscidn', 'IDcase', 'branch', 'courthousecity', 'districtOrig',
       'amountoffine', 'lifesentence', 'probationsentencemonths',
       'prisonsentencemonths', 'trial', 'chargetype', 'leadcharge', 'chg_type',
       'obsnum', 'district_code', 'fiscalyear'],
      dtype='object')
1
Index(['judgelastname', 'judgemiddlename', 'pooffice', 'usscidn', 'IDcase',
       'branch', 'courthousecity', 'districtOrig', 'amountoffine',
       'lifesentence', 'probationsentencemonths', 'prisonsentencemonths',
       'trial', 'chargetype', 'leadcharge', 'chg_type', 'obsnum',
       'district_code', 'fiscalyear', 'dojprogramcat',
       'groupedprogramcategory', 'investagency', 'deathsentence',
       'judgegeneration', 'commdate', 'retdate', 'termdate',
       'appointmentdistrict', 'supplemental_data', 'defendanttracid',
       'sentencingdatefy', 'commissioneddate', 'retirementdate',
       'terminationdate', 'date', 